In [1]:
import logging
import gc
from pathlib import Path

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
from pytorch_pretrained_bert import BertTokenizer
from pytorch_pretrained_bert.modeling import BertModel

from allennlp.modules.span_extractors import SelfAttentiveSpanExtractor, EndpointSpanExtractor

from helperbot import (
    TriangularLR, BaseBot, WeightDecayOptimizerWrapper,
    GradualWarmupScheduler
)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
def extract_target(df):
    df["Neither"] = 0
    df.loc[~(df['A-coref'] | df['B-coref']), "Neither"] = 1
    df["target"] = 0
    df.loc[df['B-coref'] == 1, "target"] = 1
    df.loc[df["Neither"] == 1, "target"] = 2
    print(df.target.value_counts())
    return df

In [3]:
df_train = pd.concat([
    pd.read_csv("../input/gap-test.tsv", delimiter="\t"),
    pd.read_csv("../input/gap-validation.tsv", delimiter="\t")
], axis=0)

In [4]:
df_test = pd.read_csv("../input/gap-development.tsv", delimiter="\t")

In [5]:
df_train = extract_target(df_train)
df_test = extract_target(df_test)

0    1105
1    1060
2     289
Name: target, dtype: int64
1    925
0    874
2    201
Name: target, dtype: int64


In [6]:
sample_sub = pd.read_csv("../input/sample_submission_stage_1.csv")
assert sample_sub.shape[0] == df_test.shape[0]

In [7]:
BERT_MODEL = 'bert-large-uncased'
CASED = False
tokenizer = BertTokenizer.from_pretrained(
    BERT_MODEL,
    do_lower_case=CASED,
    never_split = ("[UNK]", "[SEP]", "[PAD]", "[CLS]", "[MASK]")
)

In [8]:
class GAPDataset(Dataset):
    """Custom GAP Dataset class"""
    def __init__(self, df, tokenizer, labeled=True):
        self.labeled = labeled
        if labeled:
            self.y = df.target.values.astype("uint8")
        
        self.offsets, self.tokens = [], []
        for _, row in df.iterrows():
            tokens, offsets = tokenize(row, tokenizer)
            self.offsets.append(offsets)
            self.tokens.append(tokenizer.convert_tokens_to_ids(
                ["[CLS]"] + tokens + ["[SEP]"]))
        
    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        if self.labeled:
            return self.tokens[idx], self.offsets[idx], self.y[idx]
        return self.tokens[idx], self.offsets[idx], None

In [9]:
def tokenize(row, tokenizer):
    break_points = sorted(
        [
            ("A", row["A-offset"], row["A"]),
            ("B", row["B-offset"], row["B"]),
            ("P", row["Pronoun-offset"], row["Pronoun"]),
        ], key=lambda x: x[0]
    )
    tokens, spans, current_pos = [], {}, 0
    for name, offset, text in break_points:
        tokens.extend(tokenizer.tokenize(row["Text"][current_pos:offset]))
        # Make sure we do not get it wrong
        assert row["Text"][offset:offset+len(text)] == text
        # Tokenize the target
        tmp_tokens = tokenizer.tokenize(row["Text"][offset:offset+len(text)])
        spans[name] = [len(tokens), len(tokens) + len(tmp_tokens) - 1] # inclusive
        tokens.extend(tmp_tokens)
        current_pos = offset + len(text)
    tokens.extend(tokenizer.tokenize(row["Text"][current_pos:offset]))
    assert spans["P"][0] == spans["P"][1]
    return tokens, (spans["A"] + spans["B"] + [spans["P"][0]])

In [10]:
def collate_examples(batch, truncate_len=490):
    """Batch preparation.
    
    1. Pad the sequences
    2. Transform the target.
    """    
    transposed = list(zip(*batch))
    max_len = min(
        max((len(x) for x in transposed[0])),
        truncate_len
    )
    tokens = np.zeros((len(batch), max_len), dtype=np.int64)
    for i, row in enumerate(transposed[0]):
        row = np.array(row[:truncate_len])
        tokens[i, :len(row)] = row
    token_tensor = torch.from_numpy(tokens)
    # Offsets
    offsets = torch.stack([
        torch.LongTensor(x) for x in transposed[1]
    ], dim=0) + 1 # Account for the [CLS] token
    # Labels
    if len(transposed) == 2:
        return token_tensor, offsets, None
    labels = torch.LongTensor(transposed[2])
    return token_tensor, offsets, labels

In [11]:
test_ds = GAPDataset(df_test, tokenizer)
test_loader = DataLoader(
    test_ds,
    collate_fn = collate_examples,
    batch_size=128,
    num_workers=2,
    pin_memory=True,
    shuffle=False
)

In [12]:
class Head(nn.Module):
    """The MLP submodule"""
    def __init__(self, bert_hidden_size: int):
        super().__init__()
        self.bert_hidden_size = bert_hidden_size
#         self.span_extractor = SelfAttentiveSpanExtractor(bert_hidden_size)
        self.span_extractor = EndpointSpanExtractor(
            bert_hidden_size, "x,y,x*y"
        )
        self.fc = nn.Sequential(
            nn.BatchNorm1d(bert_hidden_size * 7),
            nn.Dropout(0.1),
            nn.Linear(bert_hidden_size * 7, 48),           
            nn.ReLU(),
            nn.BatchNorm1d(48),             
            nn.Dropout(0.5),
            nn.Linear(48, 3)
        )
        for i, module in enumerate(self.fc):
            if isinstance(module, (nn.BatchNorm1d, nn.BatchNorm2d)):
                nn.init.constant_(module.weight, 1)
                nn.init.constant_(module.bias, 0)
                print("Initing batchnorm")
            elif isinstance(module, nn.Linear):
                if getattr(module, "weight_v", None) is not None:
                    nn.init.uniform_(module.weight_g, 0, 1)
                    nn.init.kaiming_normal_(module.weight_v)
                    print("Initing linear with weight normalization")
                    assert model[i].weight_g is not None
                else:
                    nn.init.kaiming_normal_(module.weight)
                    print("Initing linear")
                nn.init.constant_(module.bias, 0)
                
    def forward(self, bert_outputs, offsets):
        assert bert_outputs.size(2) == self.bert_hidden_size
        spans_contexts = self.span_extractor(
            bert_outputs, 
            offsets[:, :4].reshape(-1, 2, 2)
        ).reshape(offsets.size()[0], -1)
        return self.fc(torch.cat([
            spans_contexts,
            torch.gather(
                bert_outputs, 1,
                offsets[:, [4]].unsqueeze(2).expand(-1, -1, self.bert_hidden_size)
            ).squeeze(1)
        ], dim=1))


class GAPModel(nn.Module):
    """The main model."""
    def __init__(self, bert_model: str, device: torch.device):
        super().__init__()
        self.device = device
        if bert_model in ("bert-base-uncased", "bert-base-cased"):
            self.bert_hidden_size = 768
        elif bert_model in ("bert-large-uncased", "bert-large-cased"):
            self.bert_hidden_size = 1024
        else:
            raise ValueError("Unsupported BERT model.")
        self.bert = BertModel.from_pretrained(bert_model).to(device)
        self.head = Head(self.bert_hidden_size).to(device)
    
    def forward(self, token_tensor, offsets):
        token_tensor = token_tensor.to(self.device)
        bert_outputs, _ =  self.bert(
            token_tensor, attention_mask=(token_tensor > 0).long(), 
            token_type_ids=None, output_all_encoded_layers=False)
        head_outputs = self.head(bert_outputs, offsets.to(self.device))
        return head_outputs

In [13]:
# Adapted from fast.ai library
def children(m):
    return m if isinstance(m, (list, tuple)) else list(m.children())


def set_trainable_attr(m, b):
    m.trainable = b
    for p in m.parameters():
        p.requires_grad = b


def apply_leaf(m, f):
    c = children(m)
    if isinstance(m, nn.Module):
        f(m)
    if len(c) > 0:
        for l in c:
            apply_leaf(l, f)

            
def set_trainable(l, b):
    apply_leaf(l, lambda m: set_trainable_attr(m, b))

In [14]:
class GAPBot(BaseBot):
    def __init__(self, model, train_loader, val_loader, *, optimizer, clip_grad=0,
        avg_window=100, log_dir="./cache/logs/", log_level=logging.INFO,
        checkpoint_dir="./cache/model_cache/", batch_idx=0, echo=False,
        device="cuda:0", use_tensorboard=False):
        super().__init__(
            model, train_loader, val_loader, 
            optimizer=optimizer, clip_grad=clip_grad,
            log_dir=log_dir, checkpoint_dir=checkpoint_dir, 
            batch_idx=batch_idx, echo=echo,
            device=device, use_tensorboard=use_tensorboard
        )
        self.criterion = torch.nn.CrossEntropyLoss()
        self.loss_format = "%.6f"
        
    def extract_prediction(self, tensor):
        return tensor
    
    def snapshot(self):
        """Override the snapshot method because Kaggle kernel has limited local disk space."""
        loss = self.eval(self.val_loader)
        loss_str = self.loss_format % loss
        self.logger.info("Snapshot loss %s", loss_str)
        self.logger.tb_scalars(
            "losses", {"val": loss},  self.step)
        target_path = (
            self.checkpoint_dir / "best.pth")        
        if not self.best_performers or (self.best_performers[0][0] > loss):
            torch.save(self.model.state_dict(), target_path)
            self.best_performers = [(loss, target_path, self.step)]
        self.logger.info("Saving checkpoint %s...", target_path)
        assert Path(target_path).exists()
        return loss

In [15]:
skf = StratifiedKFold(n_splits=5, random_state=191)

val_preds, test_preds, val_ys, val_losses = [], [], [], []
for train_index, valid_index in skf.split(df_train, df_train["target"]):
    print("=" * 20)
    print("Fold " + str(len(val_preds)))
    print("=" * 20)
    train_ds = GAPDataset(df_train.iloc[train_index], tokenizer)
    val_ds = GAPDataset(df_train.iloc[valid_index], tokenizer)
    train_loader = DataLoader(
        train_ds,
        collate_fn = collate_examples,
        batch_size=32,
        num_workers=2,
        pin_memory=True,
        shuffle=True,
        drop_last=True
    )
    val_loader = DataLoader(
        val_ds,
        collate_fn = collate_examples,
        batch_size=64,
        num_workers=2,
        pin_memory=True,
        shuffle=False
    )
    model = GAPModel(BERT_MODEL, torch.device("cuda:0"))
    # You can unfreeze the last layer of bert by calling set_trainable(model.bert.encoder.layer[23], True)
    set_trainable(model.bert, False)
    set_trainable(model.head, True)
    optimizer = WeightDecayOptimizerWrapper(
        torch.optim.Adam(model.parameters(), lr=2e-3),
        0.05
    )
    # optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)
    bot = GAPBot(
        model, train_loader, val_loader,
        optimizer=optimizer, echo=True,
        avg_window=40
    )
    gc.collect()
    steps_per_epoch = len(train_loader) 
    n_steps = steps_per_epoch * 15
    bot.train(
        n_steps,
        log_interval=steps_per_epoch // 2,
        snapshot_interval=steps_per_epoch,
#         scheduler=GradualWarmupScheduler(optimizer, 20, int(steps_per_epoch * 4),
#             after_scheduler=CosineAnnealingLR(
#                 optimizer, n_steps - int(steps_per_epoch * 4)
#             )
#         )
        scheduler=TriangularLR(
            optimizer, 20, ratio=3, steps_per_cycle=n_steps)
    )
    # Load the best checkpoint
    bot.load_model(bot.best_performers[0][1])
    bot.remove_checkpoints(keep=0)    
    val_preds.append(torch.softmax(bot.predict(val_loader), -1).clamp(1e-4, 1-1e-4).cpu().numpy())
    val_ys.append(df_train.iloc[valid_index].target.astype("uint8").values)
    val_losses.append(log_loss(val_ys[-1], val_preds[-1]))
    bot.logger.info("Confirm val loss: %.4f", val_losses[-1])
    test_preds.append(torch.softmax(bot.predict(test_loader), -1).clamp(1e-4, 1-1e-4).cpu().numpy())

Fold 0


[[03/22/2019 09:14:31 PM]] SEED: 9293
[[03/22/2019 09:14:31 PM]] # of paramters: 335,500,579
[[03/22/2019 09:14:31 PM]] # of trainable paramters: 358,691
[[03/22/2019 09:14:32 PM]] Optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 0.002
    lr: 0.0001
    weight_decay: 0
)
[[03/22/2019 09:14:32 PM]] Batches per epoch: 61
[[03/22/2019 09:14:32 PM]] ====================Epoch 1====================


Initing batchnorm
Initing linear
Initing batchnorm
Initing linear


[[03/22/2019 09:14:58 PM]] Step 30: train 1.584353 lr: 3.333e-04
[[03/22/2019 09:15:29 PM]] Step 60: train 1.388470 lr: 5.833e-04
100%|██████████| 8/8 [00:17<00:00,  2.02s/it]
[[03/22/2019 09:15:47 PM]] Snapshot loss 0.790815
[[03/22/2019 09:15:48 PM]] Saving checkpoint cache/model_cache/best.pth...
[[03/22/2019 09:15:48 PM]] New low

[[03/22/2019 09:15:48 PM]] ====================Epoch 2====================
[[03/22/2019 09:16:19 PM]] Step 90: train 1.236052 lr: 8.333e-04
[[03/22/2019 09:16:53 PM]] Step 120: train 1.159770 lr: 1.083e-03
100%|██████████| 8/8 [00:19<00:00,  2.31s/it]
[[03/22/2019 09:17:15 PM]] Snapshot loss 0.723812
[[03/22/2019 09:17:23 PM]] Saving checkpoint cache/model_cache/best.pth...
[[03/22/2019 09:17:23 PM]] New low

[[03/22/2019 09:17:23 PM]] ====================Epoch 3====================
[[03/22/2019 09:17:52 PM]] Step 150: train 1.087198 lr: 1.333e-03
[[03/22/2019 09:18:27 PM]] Step 180: train 1.028516 lr: 1.583e-03
100%|██████████| 8/8 [00:19<00:00,  2.30s/i

Fold 1


...

[[03/22/2019 10:30:52 PM]] Step 30: train 1.606441 lr: 3.333e-04
[[03/22/2019 10:31:26 PM]] Step 60: train 1.446130 lr: 5.833e-04
100%|██████████| 8/8 [00:17<00:00,  1.88s/it]
[[03/22/2019 10:31:44 PM]] Snapshot loss 0.893335
[[03/22/2019 10:31:45 PM]] Saving checkpoint cache/model_cache/best.pth...
[[03/22/2019 10:31:45 PM]] New low

[[03/22/2019 10:31:45 PM]] ====================Epoch 2====================
[[03/22/2019 10:32:20 PM]] Step 90: train 1.282609 lr: 8.333e-04
[[03/22/2019 10:32:53 PM]] Step 120: train 1.211982 lr: 1.083e-03
100%|██████████| 8/8 [00:17<00:00,  1.88s/it]
[[03/22/2019 10:33:13 PM]] Snapshot loss 0.719404
[[03/22/2019 10:33:21 PM]] Saving checkpoint cache/model_cache/best.pth...
[[03/22/2019 10:33:21 PM]] New low

[[03/22/2019 10:33:21 PM]] ====================Epoch 3====================
[[03/22/2019 10:33:52 PM]] Step 150: train 1.133035 lr: 1.333e-03
[[03/22/2019 10:34:27 PM]] Step 180: train 1.074981 lr: 1.583e-03
100%|██████████| 8/8 [00:17<00:00,  1.88s/i

Fold 4


[[03/22/2019 10:55:12 PM]] SEED: 9293
[[03/22/2019 10:55:12 PM]] # of paramters: 335,500,579
[[03/22/2019 10:55:12 PM]] # of trainable paramters: 358,691
[[03/22/2019 10:55:12 PM]] Optimizer Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 0.002
    lr: 0.0001
    weight_decay: 0
)
[[03/22/2019 10:55:12 PM]] Batches per epoch: 61
[[03/22/2019 10:55:12 PM]] ====================Epoch 1====================


Initing batchnorm
Initing linear
Initing batchnorm
Initing linear


[[03/22/2019 10:55:42 PM]] Step 30: train 1.525416 lr: 3.333e-04
[[03/22/2019 10:56:16 PM]] Step 60: train 1.391759 lr: 5.833e-04
100%|██████████| 8/8 [00:18<00:00,  2.34s/it]
[[03/22/2019 10:56:36 PM]] Snapshot loss 0.878458
[[03/22/2019 10:56:37 PM]] Saving checkpoint cache/model_cache/best.pth...
[[03/22/2019 10:56:37 PM]] New low

[[03/22/2019 10:56:38 PM]] ====================Epoch 2====================
[[03/22/2019 10:57:13 PM]] Step 90: train 1.249881 lr: 8.333e-04
[[03/22/2019 10:57:46 PM]] Step 120: train 1.163046 lr: 1.083e-03
100%|██████████| 8/8 [00:18<00:00,  2.34s/it]
[[03/22/2019 10:58:07 PM]] Snapshot loss 0.720036
[[03/22/2019 10:58:15 PM]] Saving checkpoint cache/model_cache/best.pth...
[[03/22/2019 10:58:15 PM]] New low

[[03/22/2019 10:58:15 PM]] ====================Epoch 3====================
[[03/22/2019 10:58:46 PM]] Step 150: train 1.105200 lr: 1.333e-03
[[03/22/2019 10:59:21 PM]] Step 180: train 1.050429 lr: 1.583e-03
100%|██████████| 8/8 [00:18<00:00,  2.34s/i

In [17]:
test_preds

[array([[0.74793327, 0.2143201 , 0.03774654],
        [0.9880679 , 0.00988253, 0.0020495 ],
        [0.02281776, 0.6646731 , 0.3125091 ],
        ...,
        [0.46640489, 0.28172353, 0.25187162],
        [0.9913982 , 0.00413572, 0.00446608],
        [0.0613956 , 0.3166686 , 0.6219358 ]], dtype=float32),
 array([[0.17300756, 0.73620236, 0.09079007],
        [0.9953689 , 0.00311402, 0.00151702],
        [0.04477216, 0.8251235 , 0.13010427],
        ...,
        [0.18542829, 0.68099666, 0.13357507],
        [0.9867958 , 0.0096134 , 0.00359084],
        [0.04641515, 0.84494936, 0.10863551]], dtype=float32),
 array([[0.66663164, 0.24300736, 0.09036099],
        [0.98768544, 0.00881832, 0.00349617],
        [0.03495398, 0.8585785 , 0.10646757],
        ...,
        [0.3258655 , 0.5802075 , 0.09392688],
        [0.99470913, 0.00337524, 0.00191567],
        [0.15055966, 0.7541962 , 0.0952441 ]], dtype=float32),
 array([[4.8624769e-01, 3.6400113e-01, 1.4975119e-01],
        [9.6186429e-01, 3.4

In [19]:
val_losses


[0.5772331131093703,
 0.5470178227560701,
 0.5792128030876269,
 0.5992271633830719,
 0.5330046828203046]

In [20]:
final_test_preds = np.mean(test_preds, axis=0)
final_test_preds.shape

(2000, 3)

In [21]:
log_loss(df_test.target, final_test_preds)

0.5343874468779977

In [22]:
# Create submission file
df_sub = pd.DataFrame(final_test_preds, columns=["A", "B", "NEITHER"])
df_sub["ID"] = df_test.ID
df_sub.to_csv("submission.csv", index=False)
df_sub.head()

,A,B,NEITHER,ID
0,0.528435,0.355731,0.115834,development-1
1,0.984943,0.012316,0.002741,development-2
2,0.025796,0.796385,0.177820,development-3
3,0.031866,0.569177,0.398956,development-4
4,0.007956,0.980349,0.011695,development-5
